In [1]:
%matplotlib inline
import pims
from pims import ND2_Reader
from ipywidgets import interact
from ipywidgets import widgets
from fnmatch import fnmatch
import numpy as np
import matplotlib.pyplot as plt
from flowdec.nb import utils as nbutils 
from flowdec import data as fd_data
# import pims
# import pims
from flowdec import restoration as fd_restoration
from flowdec import data as fd_data
from flowdec import psf as fd_psf
import dask
import dask.array as da
import tifffile as tf
from nd2reader import ND2Reader
# from pims import ND2_Reader 
# import aicsimageio.vendor.omexml as ome
import glob
import time, sys,os
sys.path.insert(0,'./libraries')
from deco_libraries import update_progress, pimsmeta2OMEXML, Nd2meta2OMEXML, observer, init_RL_algo, deconv, depth_divide
from deco_libraries import SelectFilesButton, tick_choices, get_deltaT

In [2]:
my_button = SelectFilesButton()
my_button # This will display the button in the context of Jupyter Notebook

Button(description='Select Files', icon='square-o', style=ButtonStyle(button_color='orange'))

Button(description='Select Directory', icon='square-o', style=ButtonStyle(button_color='orange'))

In [3]:
if my_button.button.pick:
    filelist = my_button.button.files
elif my_button.button2.pick:
    patterns = ("*.nd2","*.ome.tifNO")
    filelist = [f.path for f in os.scandir(my_button.button2.files) if any(fnmatch(f, p) for p in patterns)]
    filelist.sort()

In [4]:
#Load the first file for reference and what's inside the file
frames = ND2Reader(filelist[0])
print(frames.sizes)
#Grab metadata
metadata = frames.metadata
print(metadata['channels'])

#If you don't have enough VRAM divide the XY to do the deconvolution that is automatically assembled after
xdivide = round(frames.sizes['x']/1024)
ydivide = round(frames.sizes['y']/1024)
depthdivide = depth_divide(xdivide, ydivide)
frames

{'x': 2048, 'y': 2048, 't': 1}
['DAPI']


/home/jmamede/anaconda3/lib/python3.7/site-packages/nd2reader/raw_metadata.py:173: UserWarning: Z-levels details missing in metadata. Using Z-coordinates instead.
  warnings.warn("Z-levels details missing in metadata. Using Z-coordinates instead.")


<FramesSequenceND>
Axes: 3
Axis 'x' size: 2048
Axis 'y' size: 2048
Axis 't' size: 1
Pixel Datatype: <class 'numpy.float64'>

In [5]:
metadata

{'height': 2048,
 'width': 2048,
 'date': datetime.datetime(2021, 8, 25, 11, 0, 48),
 'fields_of_view': [0],
 'frames': [0],
 'z_levels': range(0, 1),
 'z_coordinates': [5612.75],
 'x_coordinates': [4596.8],
 'y_coordinates': [2695.6000000000004],
 'total_images_per_channel': 1,
 'channels': ['DAPI'],
 'pixel_microns': 0.1625,
 'num_frames': 1,
 'experiment': {'description': 'unknown', 'loops': []},
 'events': []}

In [ ]:
frames.metadata